In [1]:
#!pip install pulp
import pulp as lp
import modulo_conn_sql as mcq
import numpy as np
import pandas as pd 

In [2]:
def conectarSQL():
    conn = mcq.ConexionSQL()
    cursor = conn.getCursor()
    return cursor

#Query BD SQL-Server Cemex
def querySQL(query, parametros):
    #Conectar con base sql y ejecutar consulta
    cursor = conectarSQL()
    try:
        cursor.execute(query, parametros)
        #obtener nombre de columnas
        names = [ x[0] for x in cursor.description]
        
        #Reunir todos los resultado en rows
        rows = cursor.fetchall()
        resultadoSQL = []
            
        #Hacer un array con los resultados
        while rows:
            resultadoSQL.append(rows)
            if cursor.nextset():
                rows = cursor.fetchall()
            else:
                rows = None
                
        #Redimensionar el array para que quede en dos dimensiones
        resultadoSQL = np.array(resultadoSQL)
        resultadoSQL = np.reshape(resultadoSQL, (resultadoSQL.shape[1], resultadoSQL.shape[2]) )
    finally:
            if cursor is not None:
                cursor.close()
    return pd.DataFrame(resultadoSQL, columns = names)

#### Datos

In [3]:
# parametros SQL
pais = "Colombia"
version_forecast = 'CONSENSO_MAY_2022'

In [12]:
# Nombre Cluster
nombre_cluster = querySQL( "SELECT Centro, Ciudad_Cluster as Ciudad, [Desc Cluster] as Cluster, [Planta Unica] as PlantaUnica FROM SCAC_AT1_NombreCluster where Pais = ? and Activo = 1" , (pais) )
nombre_cluster = nombre_cluster[nombre_cluster['PlantaUnica'] != 'CO-PLANTA MACEO']

In [5]:
#Dataset de la demanda
df_forecast = querySQL( "SELECT * FROM SCAC_AV7_DesagregacionPronosticoCiudadPlantaDiaTabla WHERE Version = ? and Pais = ? " ,(version_forecast, pais) )
demanda = df_forecast.groupby(['PlantaUnica'])['M3Forecast'].sum()

In [185]:
# Otros datos
datos_modelo = pd.read_excel('./datos/Datos Modelo.xlsx')

#### Conjuntos

In [186]:
plantas = nombre_cluster['PlantaUnica'].unique()

#### Parametros

In [187]:
#demanda por planta
d = demanda.to_dict()

#capacidad de produccion
q = datos_modelo[['Plantas', 'Qi']].to_dict('list')
q = {planta: val for planta, val in zip(q['Plantas'], q['Qi']) }

#productividad diaria mixer
p = datos_modelo[['Plantas', 'Pi']].to_dict('list')
p = {planta: val for planta, val in zip(p['Plantas'], p['Pi']) }


In [164]:
#flujo de la planta
f = datos_modelo[['Plantas', 'Fi Marzo']].to_dict('list')
f = {planta: val for planta, val in zip(f['Plantas'], f['Fi Marzo']) }

In [188]:
#flujo de la planta
f = datos_modelo[['Plantas', 'Fi Año']].to_dict('list')
f = {planta: val for planta, val in zip(f['Plantas'], f['Fi Año']) }

### Modelamiento

#### Declaracion del modelo

In [189]:
modelo = lp.LpProblem(sense = lp.LpMaximize)

#### Variables de decision

In [190]:
# volumen por planta
x = lp.LpVariable.dicts('volumenes', indexs = plantas, lowBound = 0, cat = lp.LpContinuous)
y = lp.LpVariable.dicts('flota', indexs = plantas, lowBound = 0, cat = lp.LpInteger)


<ipython-input-190-654703e5d918>:2: DeprecationWarning: 'indexs' is deprecated; use 'indices'.
  x = lp.LpVariable.dicts('volumenes', indexs = plantas, lowBound = 0, cat = lp.LpContinuous)
<ipython-input-190-654703e5d918>:3: DeprecationWarning: 'indexs' is deprecated; use 'indices'.
  y = lp.LpVariable.dicts('flota', indexs = plantas, lowBound = 0, cat = lp.LpInteger)


#### Funcion Objetivo

In [191]:
# maximizar el volumen por el flujo de cada planta
modelo += lp.lpSum(f[i]*x[i] for i in plantas)

#### Restricciones

In [192]:
# el volumen de cada planta no  puede superar su capacidad
for i in plantas:
    modelo += x[i] <= q[i], "R1_"+i

In [193]:
# el volumen de cada planta no puede ser mayor a su demanda estimada
for i in plantas:
    modelo += x[i] <= d[i], "R2_"+i

In [194]:
# capacidad de mixer
for i in plantas:
    modelo += y[i] * p[i] * 25 >= x[i]

In [195]:
#Demanda Minima
modelo += x['CO-PLANTA BARRANQUILLA'] >= 9800.0
modelo += x['CO-PLANTA FLORIDA'] >= 9600.0

modelo += x['CO-PLANTA CARTAGENA'] == 7840.0
modelo += x['CO-PLANTA Paraiso Central'] == 1317
modelo += x['CO-PLANTA Canoas'] == 1037.0
modelo += x['CO-PLANTA FUSA'] == 2000.0
modelo += x['CO-PLANTA VISTA HERMOSA'] >= 2164.0

In [196]:
# Restriccion de mixers
modelo += lp.lpSum(y) <= 350

In [197]:
modelo.solve()

1

In [198]:
lp.value(modelo.objective)

3665893.425384448

In [199]:
resultados = {i : x[i].value() for i in plantas}
resultados

{'CO-PLANTA 240': 10062.5,
 'CO-PLANTA BOSA': 6327.5717,
 'CO-PLANTA SUR (DTE)': 5854.3224,
 'CO-PLANTA TOCANCIPA': 2417.0131,
 'CO-PLANTA CALI': 5100.4464,
 'CO-PLANTA CALI SUR': 5488.7795,
 'CO-PLANTA MEDELLIN': 5439.9369,
 'CO-PLANTA PEREIRA': 4225.6944,
 'CO-PLANTA BARRANQUILLA': 10000.0,
 'CO-PLANTA CUCUTA': 1816.9408,
 'CO-PLANTA CARTAGENA': 7840.0,
 'CO-PLANTA SOACHA': 5463.8916,
 'CO-PLANTA SANTA MARTA': 2902.3707,
 'CO-PLANTA PUENTE ARANDA': 15325.043,
 'CO-PLANTA CALLE 170': 8464.6552,
 'CO-PLANTA SUMAPAZ': 7099.9994,
 'CO-PLANTA SIBERIA': 8268.2153,
 'CO-PLANTA NEIVA': 3825.0,
 'CO-PLANTA FLORIDA': 9600.0096,
 'CO-PLANTA TULUA PARAJE LA RIVERA': 1600.0092,
 'CO-PLANTA VISTA HERMOSA': 2164.2202,
 'CO-PLANTA CON. IBAGUE': 9750.0,
 'CO-PLANTA RIONEGRO': 4650.0,
 'CO-PLANTA BELLO': 3646.0396,
 'CO-PLANTA MASSEQ': 700.01788,
 'CO-PLANTA La Reforma': 1378.9146,
 'CO-PLANTA FUSA': 2000.0,
 'CO-PLANTA Paraiso Central': 1317.0,
 'CO-PLANTA Canoas': 1037.0}

In [200]:
resultados_x = pd.DataFrame(list(resultados.items()),columns = ['Planta','Volumen']) 
resultados_x

,Planta,Volumen
0,CO-PLANTA 240,10062.50000
1,CO-PLANTA BOSA,6327.57170
2,CO-PLANTA SUR (DTE),5854.32240
3,CO-PLANTA TOCANCIPA,2417.01310
4,CO-PLANTA CALI,5100.44640
5,CO-PLANTA CALI SUR,5488.77950
6,CO-PLANTA MEDELLIN,5439.93690
7,CO-PLANTA PEREIRA,4225.69440
8,CO-PLANTA BARRANQUILLA,10000.00000
9,CO-PLANTA CUCUTA,1816.94080


In [201]:
resultados = {i : y[i].value() for i in plantas}
resultados_y = pd.DataFrame(list(resultados.items()),columns = ['Planta','Flota']) 
resultados_y

,Planta,Flota
0,CO-PLANTA 240,23.0
1,CO-PLANTA BOSA,17.0
2,CO-PLANTA SUR (DTE),14.0
3,CO-PLANTA TOCANCIPA,5.0
4,CO-PLANTA CALI,11.0
5,CO-PLANTA CALI SUR,12.0
6,CO-PLANTA MEDELLIN,13.0
7,CO-PLANTA PEREIRA,10.0
8,CO-PLANTA BARRANQUILLA,21.0
9,CO-PLANTA CUCUTA,4.0


In [202]:
resultados_df = pd.merge(resultados_x, resultados_y, on = 'Planta')
resultados_df

,Planta,Volumen,Flota
0,CO-PLANTA 240,10062.50000,23.0
1,CO-PLANTA BOSA,6327.57170,17.0
2,CO-PLANTA SUR (DTE),5854.32240,14.0
3,CO-PLANTA TOCANCIPA,2417.01310,5.0
4,CO-PLANTA CALI,5100.44640,11.0
5,CO-PLANTA CALI SUR,5488.77950,12.0
6,CO-PLANTA MEDELLIN,5439.93690,13.0
7,CO-PLANTA PEREIRA,4225.69440,10.0
8,CO-PLANTA BARRANQUILLA,10000.00000,21.0
9,CO-PLANTA CUCUTA,1816.94080,4.0


In [203]:
writer = pd.ExcelWriter("./datos/Resultados_modelo_" + pais + "_" + pd.to_datetime("now").strftime("%Y%m%d%H%M%S") + ".xlsx", engine='xlsxwriter')
resultados_df.to_excel(writer, sheet_name="Flujo Año", index=False )
writer.save()
